In [1]:
import pandas as pd
import string
import re

# #from attr.filters import exclude
# from gradio.themes.builder_app import suggestions
# from tensorflow_io.python.ops.bigtable.bigtable_row_range import prefix


In [2]:
df = pd.read_csv(r"C:\Users\Asus\NLP\IMDB Dataset.csv\IMDB Dataset.csv")

In [3]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:

df['review'] = df['review'].str.lower()

In [5]:
def text_to_dict(filename, delimiter='='):
    my_dict = {}
    
    with open(filename, 'r') as file:
        for line in file:
            # Strip leading/trailing whitespace and split by the delimiter
            parts = line.strip().split(delimiter)
            
            # Check if we have exactly two parts after splitting
            if len(parts) == 2:
                key = parts[0].strip()
                value = parts[1].strip()
                my_dict[key] = value
    
    return my_dict

filename = r'C:\Users\Asus\NLP\IMDB Dataset.csv\slang.txt'
chat_words_dict = text_to_dict(filename)


In [6]:
def remove_urls(text):
    pattern = re.compile(r'https?://\S+|www\.\S+.*?>')
    return pattern.sub(r'',text)

def remove_html(text):
    pattern = re.compile(r'<.*?>')
    return pattern.sub(r'',text)

exclude = string.punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans('','',exclude))

def chat_word_treatment(text):
    arr = []
    for i in text.split():
        if i.upper() in chat_words_dict:
            arr.append(chat_words_dict[i.upper()])
        else:
            arr.append(i)
            
    return " ".join(arr)


In [7]:
def all_function(x):
    for func in function:
        x= func(x)
        return x

function = [remove_html,remove_urls,remove_punctuation,chat_word_treatment]
df['review']=df['review'].apply(all_function)

In [8]:
df.sample(5)

,review,sentiment
49070,"""brotherhood of satan"" is one of the most unde...",positive
4836,this movie is definently a horror movie and if...,positive
33874,this tv film tells the story of extrovert fran...,negative
7592,what the bleep do we know is a deluded and hap...,negative
41207,i hoped for this show to be somewhat realistic...,negative


# symspell (libabry) used for spelling correction!!!

In [9]:
from symspellpy import SymSpell, Verbosity

sym_spell = SymSpell(max_dictionary_edit_distance=2,prefix_length=7)
sym_spell.load_dictionary(r"C:\Users\Asus\NLP\twitter sentiment\symspelling_dict\frequency_dictionary_en_82_765.txt",term_index=0,count_index=0)

def correct_spell(sentence):
    words = sentence.split()
    
    corrected = []
    
    for word in words:
        
        suggestions = sym_spell.lookup(word,Verbosity.CLOSEST,max_edit_distance= 2)
        
        if suggestions:
            corrected.append(suggestions[0].term)
        else:
            corrected.append(word)
            
    return ' '.join(corrected)

In [10]:
df['review']=df['review'].apply(correct_spell)

In [11]:
df.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production. the filming tec...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there's a family where a little boy ...,negative
4,"petter mattei's ""love in the time of money"" is...",positive


# Stop words

In [12]:
import nltk
# nltk.download("stopwords")
from nltk.corpus import stopwords

In [13]:
set(stopwords.words('english'))

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [14]:

stop_words_list = set(stopwords.words('english'))
def stop_words(texts):
    
    new_text=[]
    for word in texts.split():
        if word in stop_words_list:
            new_text.append('')
        else:
            new_text.append(word)
    return " ".join(new_text)

df['review']=df['review'].apply(stop_words)
      

# Tokenization

In [17]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [18]:
from nltk.tokenize import word_tokenize, sent_tokenize


In [28]:
sent ="""kjsdlvsz
vssdvsjdv djdnsd i am going sd sd ks dvsd. vks dvk sdk. sdkv sd vksd vds kj alj cad"""
word = 'i am going to dehli!'
word2 = "a 5km ride cost $10.50"
word3 = "we're here to help! mail us at nks@gmail.com"
word_tokenize(word)

['i', 'am', 'going', 'to', 'dehli', '!']

In [29]:
sent_tokenize(sent)

['kjsdlvsz\nvssdvsjdv djdnsd i am going sd sd ks dvsd.',
 'vks dvk sdk.',
 'sdkv sd vksd vds kj alj cad']

In [30]:
word_tokenize(word2)

['a', '5km', 'ride', 'cost', '$', '10.50']

In [31]:
word_tokenize(word3)

['we',
 "'re",
 'here',
 'to',
 'help',
 '!',
 'mail',
 'us',
 'at',
 'nks',
 '@',
 'gmail.com']

# Spacy tokenization lib

In [42]:
from typing import TypeAlias

import spacy

ImportError: cannot import name 'TypeAliasType' from 'typing_extensions' (C:\Users\Asus\AppData\Local\Programs\Python\Python311\Lib\site-packages\typing_extensions.py)

In [40]:
from typing import Annotated, Final, Literal, TypeAlias, TypeGuard, get_args, get_origin